<a href="https://colab.research.google.com/github/hwon-n/cp2/blob/hyewon/CP2_NSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bert

  Created wheel for bert: filename=bert-2.2.0-py3-none-any.whl size=3765 sha256=a2593af00cc0051902dd744c07160ad7569babf38d3c2308784c018f10b7512b
  Stored in directory: /root/.cache/pip/wheels/bb/31/1b/c05f362e347429b7436954d1a2280fe464731e8f569123a848
  Created wheel for erlastic: filename=erlastic-2.0.0-py3-none-any.whl size=6794 sha256=4a5da566fdd67886ea8d068deecc38c0c0af0e06e83e2324a615c4c1657b5c5a
  Stored in directory: /root/.cache/pip/wheels/94/f1/b4/0b98b1e94775da6a0b1130e342d22af05cd269e1172c19f40f
Successfully built bert erlastic


In [3]:
!pip install mxnet-cu101

     |████████████████████████████████| 358.1 MB 5.2 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [4]:
!pip install gluonnlp tqdm

     |████████████████████████████████| 344 kB 7.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595740 sha256=2da83dce4547de3075cbe477f0fcd2517734a8c2b3570d2d220d9e8cdc9b2f9a
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp


In [5]:
!pip install mxnet

     |████████████████████████████████| 47.3 MB 135 kB/s 


In [6]:
pip install transformers

     |████████████████████████████████| 3.8 MB 7.9 MB/s 
     |████████████████████████████████| 6.5 MB 53.2 MB/s 
     |████████████████████████████████| 67 kB 6.9 MB/s 
     |████████████████████████████████| 596 kB 51.5 MB/s 
     |████████████████████████████████| 895 kB 73.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# 데이터셋 준비

In [ ]:
import os

root = '/content/drive/MyDrive/news_class9x13000/'
categories = os.listdir(root)

dataset = []

for category in categories:
  files = os.listdir(root + category)
  for i, f in enumerate(files):
    fname = root + category + '/' + f
    file = open(fname, 'r', encoding='utf-8')
    strings = file.read()
    dataset.append([strings])
    file.close()
    if i >= 199:
      break
  

In [ ]:
len(dataset)

1800

In [ ]:
table = str.maketrans({
    '\'': '',
    '\"': '',
    '★': '',
    '▲': '',
    '▶': '',
    '┌': '',
    '├': '',
    '│': '',
    '└': '',
    '→': '',
    '‘': '',
    '■': '',
    '○': ''

})

In [ ]:
process_dataset = []

for sentences in dataset:
  sentences = sentences[0]
  sentences = sentences.translate(table)
  sentences = sentences.split('\n')
  del sentences[0]
  process_dataset.extend(sentences)

process_dataset

In [ ]:
len(process_dataset)

62787

In [ ]:
import csv

with open(
    os.path.join(root, 'process data.tsv'), mode = 'w', encoding = 'utf-8', newline = ''
) as f:
  for data in process_dataset:
    tw = csv.writer(f, delimiter = '\t')
    tw.writerow(data)

print('save the preprocessing data (del title version) file')

save the preprocessing data (del title version) file


In [ ]:
train_dataset = process_dataset[:15000]
test_dataset = process_dataset[15000:25000]

len(train_dataset), len(test_dataset)

(15000, 10000)

In [ ]:
train_dataset[14999]

'아내가 지혜로우면 남편은 바보이다.'

In [ ]:
test_dataset[9999]

'끝에까지 다 볼 수 있다면서 너무 많은 경우의 줄이기 위해 유럽 아마추어 고수들의 서로 대국한 입력 자료를 프로 기사들 것으로 업그레이드해서, 똑같은 과정을 그대로 해 이기는 방법을 배웠다며 알파고 진화 과정을 설명했다.'

In [ ]:
def save_file(dataset, fname):
  with open(
    os.path.join(root, fname), mode = 'w', encoding = 'utf-8', newline = ''
  ) as f:
    for data in process_dataset:
      tw = csv.writer(f, delimiter = '\t')
      tw.writerow(data)
  
save_file(train_dataset, 'train.tsv')
save_file(test_dataset, 'test.tsv')

# 데이터 만들기

    corpus = [prompt, next, isNext]

    for index in range(process_dataset):
      if len(corpus) % 2 == 0: 
        이어지는 문장
        corpus.append([process_dataset[index - 1], process_dataset[index], True])
      else: 
        이어지지 않는 문장
        randomIndex = randint(0, len(corpus)-1)
        corpus.append([process_dataset[randomIndex], process_dataset[index], False])





In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Softmax
from transformers import TFBertForNextSentencePrediction
from transformers import AutoTokenizer

In [ ]:
# 모델과 tokenizer 준비

model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt = True)
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
def same_sentence(prompt, next):
  encoding = tokenizer(prompt, next, return_tensors='tf')
  logits = model(encoding['input_ids'], token_type_ids = encoding['token_type_ids'])[0]
  softmax = Softmax()
  probs = softmax(logits)
  res = tf.math.argmax(probs, axis=-1).numpy()

  return res

In [ ]:
from random import randint

def new_corpus(dataset):
  corpus = []

  for index in range(0, len(dataset)-1):
    if len(corpus) % 2 == 0:
      # 이어지는 문장
      prompt = dataset[index]
      next = dataset[index + 1]
      if same_sentence(prompt, next) == 0:
        corpus.append([1, prompt, next])
      else: 
        corpus.append([0, prompt, next])
    else:
      # 이어지지 않는 문장
      random_index = randint(0, len(dataset)-1)
      corpus.append([0, dataset[index], dataset[random_index]])
  
  return corpus

train_corpus = new_corpus(train_dataset)
test_corpus = new_corpus(test_dataset)

len(train_corpus), len(test_corpus)

(14999, 9999)

In [ ]:
import pandas as pd

train_pd = pd.DataFrame(train_corpus, columns = ['prompt', 'next', 'isNext'])
test_pdt = pd.DataFrame(test_corpus, columns = ['prompt', 'next', 'isNext'])

train_pd.head(5)

,prompt,next,isNext
0,프로크루스테스의 침대라는 그리스신화에 나오는 이야기가 있다.,이 이야기는 프로크루스테스라는 사람이 지나가는 나그네를 자신의 집으로 잡아와서 침대...,1
1,이 이야기는 프로크루스테스라는 사람이 지나가는 나그네를 자신의 집으로 잡아와서 침대...,"또, 안전행정부와 기획재정부는 전 부처와 공공기관을 대상으로 여름휴가 하루 더 가기...",0
2,결국 침대에 사람을 맞춘다는 얘기다.,올해부터 실시하고 있는 활동보조서비스 개악 지침도 이것과 다르지 않게 잔인하다.,0
3,올해부터 실시하고 있는 활동보조서비스 개악 지침도 이것과 다르지 않게 잔인하다.,흥선대원군은 이홍장의 심문을 받은 후 바오딩(保定)으로 옮겨져 4년간 중국에서 불우...,0
4,장애인활동보조서비스는 장애로 인해 일상생활의 어려움을 겪는 사람들의 원활한 일상생활...,2007년부터 시작한 이 서비스 제도는 전국의 중증장애인들이 국가를 상대로 치열한 ...,1


In [ ]:
train_pd.isNext.value_counts()

0    8390
1    6609
Name: isNext, dtype: int64

In [ ]:
test_pdt.isNext.value_counts()

0    5480
1    4519
Name: isNext, dtype: int64

In [ ]:
import csv

def save_file(fname, dataset):
  with open(os.path.join(root, fname), 'w', encoding='utf-8', newline='') as f:
    tw = csv.writer(f, delimiter='\t')
    for data in dataset:
      tw.writerow(data)

save_file('train.tsv', train_corpus)
save_file('test.tsv', test_corpus)

# Fine tuning

[참고](https://nlp.gluon.ai/examples/sentence_embedding/bert.html)

In [7]:
import warnings
warnings.filterwarnings('ignore')
import torch
import io
import random
import numpy as np
import mxnet as mx
import gluonnlp as nlp
from gluonnlp.calibration import BertLayerCollector

In [8]:
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)

In [13]:
ctx = mx.cpu()

In [23]:
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12',
                                             dataset_name='wiki_multilingual_cased',
                                             pretrained=True, ctx=ctx, use_pooler=True,
                                             use_decoder=False, use_classifier=False)

Vocab file is not found. Downloading.


In [24]:
bert_classifier = nlp.model.BERTClassifier(bert_base, num_classes=2, dropout=0.1)
# only need to initialize the classifier layer.
bert_classifier.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
bert_classifier.hybridize(static_alloc=True)

# softmax cross entropy loss for classification
loss_function = mx.gluon.loss.SoftmaxCELoss()
loss_function.hybridize(static_alloc=True)

metric = mx.metric.Accuracy()

In [25]:
# load data

train = io.open('/content/drive/MyDrive/news_class9x13000/train.tsv', encoding = 'utf-8')

for i in range(5):
  print(train.readline())


1	프로크루스테스의 침대라는 그리스신화에 나오는 이야기가 있다.	이 이야기는 프로크루스테스라는 사람이 지나가는 나그네를 자신의 집으로 잡아와서 침대에 눕혀놓고 나그네의 키가 침대보다 길면 발목을 자르고 침대보다 짧으면 다리를 강제로 늘린다는 내용을 담고 있다.

0	이 이야기는 프로크루스테스라는 사람이 지나가는 나그네를 자신의 집으로 잡아와서 침대에 눕혀놓고 나그네의 키가 침대보다 길면 발목을 자르고 침대보다 짧으면 다리를 강제로 늘린다는 내용을 담고 있다.	또, 안전행정부와 기획재정부는 전 부처와 공공기관을 대상으로 여름휴가 하루 더 가기를 권하고 있다.

0	결국 침대에 사람을 맞춘다는 얘기다.	올해부터 실시하고 있는 활동보조서비스 개악 지침도 이것과 다르지 않게 잔인하다.

0	올해부터 실시하고 있는 활동보조서비스 개악 지침도 이것과 다르지 않게 잔인하다.	흥선대원군은 이홍장의 심문을 받은 후 바오딩(保定)으로 옮겨져 4년간 중국에서 불우한 유폐생활을 하게 된다.

1	장애인활동보조서비스는 장애로 인해 일상생활의 어려움을 겪는 사람들의 원활한 일상생활과 사회활동을 돕는 인력파견 장애인복지서비스다.	2007년부터 시작한 이 서비스 제도는 전국의 중증장애인들이 국가를 상대로 치열한 투쟁을 통해 획득한 결과물이다.



In [47]:
# Split fields by tabs
field_separator = nlp.data.Splitter('\t')
# Fields to select from the file
field_indices = [1, 2, 0]
data_train_raw = nlp.data.TSVDataset(filename='/content/drive/MyDrive/news_class9x13000/train.tsv',
                                 field_separator=field_separator,
                                 num_discard_samples=0,
                                 field_indices=field_indices)

sample_id = 1
# 문장 A
print(data_train_raw[sample_id][0])
# 문장 B
print(data_train_raw[sample_id][1])
# 1 연결되는 문장, 0 연결되지 않는 문장
print(data_train_raw[sample_id][2])

이 이야기는 프로크루스테스라는 사람이 지나가는 나그네를 자신의 집으로 잡아와서 침대에 눕혀놓고 나그네의 키가 침대보다 길면 발목을 자르고 침대보다 짧으면 다리를 강제로 늘린다는 내용을 담고 있다.
또, 안전행정부와 기획재정부는 전 부처와 공공기관을 대상으로 여름휴가 하루 더 가기를 권하고 있다.
0


In [27]:
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and DMLC.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""BERT dataset transform."""

from __future__ import absolute_import

__all__ = ['BERTDatasetTransform']

import numpy as np
from gluonnlp.data import BERTSentenceTransform

class BERTDatasetTransform:
    """Dataset transformation for BERT-style sentence classification or regression.
    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    labels : list of int , float or None. defaults None
        List of all label ids for the classification task and regressing task.
        If labels is None, the default task is regression
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    label_dtype: int32 or float32, default float32
        label_dtype = int32 for classification task
        label_dtype = float32 for regression task
    """

    def __init__(self,
                 tokenizer,
                 max_seq_length,
                 class_labels=None,
                 label_alias=None,
                 pad=True,
                 pair=True,
                 has_label=True):
        self.class_labels = class_labels
        self.has_label = has_label
        self._label_dtype = 'int32' if class_labels else 'float32'
        if has_label and class_labels:
            self._label_map = {}
            for (i, label) in enumerate(class_labels):
                self._label_map[label] = i
            if label_alias:
                for key in label_alias:
                    self._label_map[key] = self._label_map[label_alias[key]]
        self._bert_xform = BERTSentenceTransform(
            tokenizer, max_seq_length, pad=pad, pair=pair)

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.
        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
          sequence or the second sequence.
        - generate valid length
        For sequence pairs, the input is a tuple of 3 strings:
        text_a, text_b and label.
        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
            label: '0'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens:  '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14
            label: 0
        For single sequences, the input is a tuple of 2 strings: text_a and label.
        Inputs:
            text_a: 'the dog is hairy .'
            label: '1'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a:  '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7
            label: 1
        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 3 strings:
            (text_a, text_b, label). For single sequences, the input is a tuple
            of 2 strings: (text_a, label).
        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)
        np.array: classification task: label id in 'int32', shape (batch_size, 1),
            regression task: label in 'float32', shape (batch_size, 1)
        """
        if self.has_label:
            input_ids, valid_length, segment_ids = self._bert_xform(line[:-1])
            label = line[-1]
            # map to int if class labels are available
            if self.class_labels:
                label = self._label_map[label]
            label = np.array([label], dtype=self._label_dtype)
            return input_ids, valid_length, segment_ids, label
        else:
            return self._bert_xform(line)

In [46]:
#tokenizer 
bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=False)

max_len = 128
all_labels = ['0', '1']
pair = True

transform = BERTDatasetTransform(bert_tokenizer, max_len,
                                                class_labels=all_labels,
                                                has_label=True,
                                                pad=True,
                                                pair=pair)

data_train = data_train_raw.transform(transform)

In [37]:
print('vocabulary used for tokenization = \n%s'%vocabulary)
print('%s token id = %s'%(vocabulary.padding_token, vocabulary[vocabulary.padding_token]))
print('%s token id = %s'%(vocabulary.cls_token, vocabulary[vocabulary.cls_token]))
print('%s token id = %s'%(vocabulary.sep_token, vocabulary[vocabulary.sep_token]))
print('token ids = \n%s'%data_train[sample_id][0])
print('segment ids = \n%s'%data_train[sample_id][1])
print('valid length = \n%s'%data_train[sample_id][2])
print('label = \n%s'%data_train[sample_id][3])

vocabulary used for tokenization = 
Vocab(size=119547, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
[PAD] token id = 1
[CLS] token id = 2
[SEP] token id = 3
token ids = 
[  2   0   0   0   0   0   0 119   3   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 119   3
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1]
segment ids = 
36
valid length = 
[0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [43]:
# 파인튜닝

batch_size = 32
lr = 5e-6

# The FixedBucketSampler and the DataLoader for making the mini-batches
train_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[1]) for item in data_train],
                                            batch_size=batch_size,
                                            shuffle=True)
bert_dataloader = mx.gluon.data.DataLoader(data_train, batch_sampler=train_sampler)

trainer = mx.gluon.Trainer(bert_classifier.collect_params(), 'adam',
                           {'learning_rate': lr, 'epsilon': 1e-9})

params = [p for p in bert_classifier.collect_params().values() if p.grad_req != 'null']
grad_clip = 1

In [44]:
# Training the model with only three epochs
log_interval = 4
num_epochs = 3
for epoch_id in range(num_epochs):
    metric.reset()
    step_loss = 0
    for batch_id, (token_ids, segment_ids, valid_length, label) in enumerate(bert_dataloader):
        with mx.autograd.record():

            # Load the data to the GPU
            token_ids = token_ids.as_in_context(ctx)
            valid_length = segment_ids.as_in_context(ctx)
            segment_ids = valid_length.as_in_context(ctx)
            label = label.as_in_context(ctx)

            # Forward computation
            out = bert_classifier(token_ids, segment_ids, valid_length.astype('float32'))
            ls = loss_function(out, label).mean()

        # And backwards computation
        ls.backward()

        # Gradient clipping
        trainer.allreduce_grads()
        nlp.utils.clip_grad_global_norm(params, 1)
        trainer.update(1)

        step_loss += ls.asscalar()
        metric.update([label], [out])

        # Printing vital information
        if (batch_id + 1) % (log_interval) == 0:
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.7f}, acc={:.3f}'
                         .format(epoch_id, batch_id + 1, len(bert_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, metric.get()[1]))
            step_loss = 0

infer_shape error. Arguments:
  data0: (32, 128)
  data1: (32,)
  data2: (32,)


ValueError: ignored